In [ ]:
#!/usr/bin/env python3

# subscriber.py
import paho.mqtt.client as mqtt
from datetime import datetime, date
from time import localtime, strftime
now = strftime("%a, %d %b %Y %H:%M:%S +0000", localtime())

# create connection, the three parameters are broker address, broker port number, and keep-alive time respectively
brokers = {"mosquitto" :"test.mosquitto.org",
           "emqx" : "broker.emqx.io",   # topic:"test" to receive automatic msg
           "pi3_wifi" :"192.168.1.57",
           "pi3_lan" : "192.168.1.100",
           "wan1" : "77.134.40.243",
           "pi4" :"192.168.0.175"}

# connect to broker
broker_name = "pi4" 
broker = brokers[broker_name]  
# print(broker)
topic = "temp_humidity"

def on_connect(client, userdata, flags, rc):
#     print(f"Connected with result code {rc}")
#     print("CTRL-Z to stop")
#     print("Subscribing with topic '" + topic + "' to " + broker_name+ ":" + broker)
    # subscribe, which need to put into on_connect
    # if reconnect after losing the connection with the broker, it will continue to subscribe to the raspberry/topic topic
    client.subscribe(topic)

# the callback function, it will be triggered when receiving messages
def on_message(client, userdata, msg):
    #print(f"{msg.topic} {msg.payload}")
    temp, humidity = bytes_to_int(msg.payload)
    #now = strftime("%a, %d %b %Y %H:%M:%S +0000", localtime())
    s  = datetime.now().isoformat()+','+str(temp)+','+str(humidity)
    
    #print(str(temp),str(humidity))
    if temp < 50 :
        print('\r',s,end='')
        #print('min',10,'Temp',str(temp),'Hum',str(humidity),end='\r')
    #client.publish("ToHost/RaspberryPi", "Hello back")

def bytes_to_int(bytes):
    l = bytes.decode("utf-8").split(',')
    return list(float(v) for v in l)

def main():
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message
    client.connect(broker, 1883, 60)  
    client.loop_forever()


In [ ]:
main()